In [1]:
import numpy as np
import pandas as pd

###  Четыре "C" очистки данных: Correcting (коррекция), Completing (заполнение/дополнение), Creating (создание), and Converting (преобразование)

In this stage, we will clean our data by 1) correcting aberrant values and outliers, 2) completing missing information, 3) creating new features for analysis, and 4) converting fields to the correct format for calculations and presentation.

- **Correcting**: Обзор данных, которые значительно отклоняются или недопустимы.
- **Completing**: В данных присутствуют пропущенные значения. Есть два метода - либо удалить записи с пропущенными значениями, либо заполнить пропущенные значения разумными значениями. Не рекомендуется удалять записи, только в самом крайнем случае. Лучший вариант - заполнить пропуски. Базовая методология - это заполнение с использованием моды, медианы или среднего + рандомизированное стандартное отклонение. Более продвинутые методоы - использовать базовую методологию основываясь на определенных критериях, например средний возраст по классы или порт по транспортным расходам и полу. Есть и более сложные методы, но перед использованием нужно сравнивать их с базовой моделью, чтобы можно было увидеть приносят ли они пользу результату. For this dataset, age will be imputed with the median, the cabin attribute will be dropped, and embark will be imputed with mode. Subsequent model iterations may modify this decision to determine if it improves the model’s accuracy.
- **Creating**: Фича инжиниринг это использование существующих фич для создания новых для того чтобы определить создают ли они новые сигналы для лучшего предсказания. For this dataset, we will create a title feature to determine if it played a role in survival.
- **Converting**: Конвертация может понадобиться для дат либо типов столбцов датасета. В датасете категориальные данные импортированы как objects, и их невозможно использовать в мат. расчетах. Преобразуем object datatypes в категориальные dummy переменные.

In [2]:
train_data = pd.read_csv("data/train.csv", index_col="PassengerId")
test_data = pd.read_csv("data/test.csv", index_col="PassengerId")
print(f"Shape of train data: {train_data.shape}. Shape of test data: {test_data.shape}")

Shape of train data: (891, 11). Shape of test data: (418, 10)


### Удаляем лишние столбцы

In [3]:
columns_to_remove = ['Cabin', 'Ticket']
train_data.drop(columns_to_remove, axis=1, inplace=True)
test_data.drop(columns_to_remove, axis=1, inplace=True)

### Удаляем выбросы

### Заполняем пропущенные значения

In [4]:
train_data['Embarked'].fillna("NAN", inplace=True)
test_data['Embarked'].fillna("NAN", inplace=True)
# Можно попробовать использовать вместо NaN моду

mean_age = train_data['Age'].mean()
median_age = train_data['Age'].median()
print(f'Среднее значение возраста: {mean_age}. Медиана: {median_age}')

train_data['Age'].fillna(median_age, inplace=True)
test_data['Age'].fillna(median_age, inplace=True)

mean_fare = test_data['Fare'].mean()
median_fare = test_data['Fare'].median()
print(f'Среднее значение затрат: {mean_fare}. Медиана: {median_fare}')
test_data['Fare'].fillna(mean_fare, inplace=True)

Среднее значение возраста: 29.69911764705882. Медиана: 28.0
Среднее значение затрат: 35.6271884892086. Медиана: 14.4542


In [5]:
print(f'Пропуски данных в трейн датасете:\n{train_data.isnull().sum()}\n' + "-"*30)
print(f'Пропуски данных в тестовом датасете:\n{test_data.isnull().sum()}\n' + "-"*30)

Пропуски данных в трейн датасете:
Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
------------------------------
Пропуски данных в тестовом датасете:
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
------------------------------


### Создаем новые характеристики (Feature engineering)

In [6]:
for dataset in [train_data, test_data]:
    # размер семьи
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    dataset['IsAlone'] = 1 #initialize to yes/1 is alone
    dataset['IsAlone'].loc[dataset['FamilySize'] > 1] = 0
    # быстрый и грубый способ отделить титул от имени
    dataset['Title'] = dataset['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]

    #Continuous variable bins; qcut vs cut: https://stackoverflow.com/questions/30211923/what-is-the-difference-between-pandas-qcut-and-pandas-cut
    #Fare Bins/Buckets using qcut or frequency bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.qcut.html
    #https://pbpython.com/pandas-qcut-cut.html
    dataset['FareBin'] = pd.qcut(dataset['Fare'], 4)

    #Age Bins/Buckets using cut or value bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.cut.html
    dataset['AgeBin'] = pd.cut(dataset['Age'].astype(int), 5)

C:\Users\nikita\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
# Очистим редкие титулы
# print(train_data['Title'].value_counts())

# stat_min = 10 #while small is arbitrary, we'll use the common minimum in statistics: http://nicholasjjackson.com/2012/03/08/sample-size-is-10-a-magic-number/
# # Создаем true-false Series с title в качестве индекса
# title_names = (train_data['Title'].value_counts() < stat_min)

# # Используем лямбду чтобы заменить титулы, которых мало на 'Misc'
# train_data['Title'] = train_data['Title'].apply(lambda x: 'Misc' if title_names.loc[x] == True else x)
# # test_data['Title'] = test_data['Title'].apply(lambda x: 'Misc' if title_names.loc[x] == True else x)
# print(train_data['Title'].value_counts())
# print(test_data['Title'].value_counts())

In [8]:
train_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,Title,FareBin,AgeBin
PassengerId,,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,2,0,Mr,"(-0.001, 7.91]","(16.0, 32.0]"
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,2,0,Mrs,"(31.0, 512.329]","(32.0, 48.0]"
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,1,1,Miss,"(7.91, 14.454]","(16.0, 32.0]"
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,2,0,Mrs,"(31.0, 512.329]","(32.0, 48.0]"
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,1,1,Mr,"(7.91, 14.454]","(32.0, 48.0]"


### Сохраняем очищенные данные в файл

In [9]:
train_data.to_csv('data/cleaned_train_data.csv', header=True)
test_data.to_csv('data/cleaned_test_data.csv', header=True)